# Swaging the database

¡Este notebook ya no es relevante! A new, cleansed dataset has been provided. Only indexing stays.

-------------

## Prelude

```sql
select count(*) from flight;
```

$52\,628\,832$

`flight` table size: $6.3\ GB$

```sql
select count(*) from flight where actual_departure is null;
```

$29\,062\,083$

```sql
delete from flight where actual_departure is null;
```

```sql
select count(*) from flight
group by carrier, flight_number, scheduled_departure;
```

`Time: 988869.873 ms  # 16.5 minutes`

## BigQuery

```sql
select count(*) from delays.flight where actual_departure is null;
```

```
Query complete (2.2s elapsed, 180 MB processed)
```

$29\,062\,083$


```sql
select count(*) from delays.flight
group by carrier, flight_number, scheduled_departure;
```

```
Query complete (105.8s elapsed, 0 B processed)
```

$21\,878\,721$ rows found.

## Indexing

```
$ time alembic upgrade head
INFO  [alembic.runtime.migration] Context impl PostgresqlImpl.
INFO  [alembic.runtime.migration] Will assume transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade 23c1e61027c7 -> 75b00c5b47fc, Indexes.

real    42m9.216s
user    0m0.832s
sys     0m0.152s
```

```sql
select count(*) from flight
group by carrier, flight_number, scheduled_departure;
```

$19\,736\,657$ rows

`Time: 934106.083 ms  # 15.5 minutes`

## Group indexing

```
$ time alembic upgrade head
INFO  [alembic.runtime.migration] Context impl PostgresqlImpl.
INFO  [alembic.runtime.migration] Will assume transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade 75b00c5b47fc -> bf308cef7025, Group index.

real    15m18.342s
user    0m0.724s
sys     0m0.096s
```

```sql
select count(*) from flight
group by carrier, flight_number, scheduled_departure;
```

$19\,736\,657$ rows

`Time: 29132.511 ms  # 0.5 minutes, we're rolling`

## Uniquity forced

```sql
select count(*) from flight;
```

$23\,566\,749$ rows


```sql
select id,
       min(actual_departure - scheduled_departure)
over (partition by carrier, flight_number, scheduled_departure)
from flight;
```

$23\,566\,749$ rows

`Time: 149843.118 ms  # 2.5 minutes`

```sql
select f2.id
from (
    select
        carrier, flight_number, scheduled_departure,
        min(actual_departure - scheduled_departure) as delay
    from flight
    group by carrier, flight_number, scheduled_departure
) as f1
inner join flight as f2
    on f2.carrier = f1.carrier
    and f2.flight_number = f1.flight_number
    and f2.scheduled_departure = f1.scheduled_departure
    and f2.actual_departure - f2.scheduled_departure = f1.delay;
```

$23\,409\,903$ rows

`Time: 225293.041 ms  # 3.75 minutes`

```sql
select f2.id
from (
    select
        carrier, flight_number, scheduled_departure,
        min(actual_departure - scheduled_departure) as delay
    from flight
    group by carrier, flight_number, scheduled_departure
) as f1
inner join flight as f2
    on f2.carrier = f1.carrier
    and f2.flight_number = f1.flight_number
    and f2.scheduled_departure = f1.scheduled_departure
    and f1.scheduled_departure + f1.delay = f2.actual_departure;
```

$23\,409\,903$ rows

`Time: 221725.877 ms  # 3.7 minutes`

```sql
select distinct f2.id
from (
    select
        carrier, flight_number, scheduled_departure,
        min(actual_departure - scheduled_departure) as delay
    from flight
    group by carrier, flight_number, scheduled_departure
) as f1
inner join flight as f2
    on f2.carrier = f1.carrier
    and f2.flight_number = f1.flight_number
    and f2.scheduled_departure = f1.scheduled_departure
    and f1.scheduled_departure + f1.delay = f2.actual_departure;
```

$23\,409\,903$ rows

`Time: 312214.387 ms  # 5.2 minutes`

```sql
select count(*) from flight
group by carrier, flight_number, scheduled_departure, actual_departure;
```

$19\,887\,747$ rows

`Time: 949011.815 ms  # 15.8 minutes`